In [91]:
import pandas as pd
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()


df = pd.read_excel("real_estate_data.xlsx",'Real_Estate_data')

df.corr() #get the correlation matrix to see to which level the dataset attributes are correlated, that will affect selecting attributes in building the forecasting model
# the matrix inform us that there is a strong relationship between pricem age, area, and floor
#df.corr()['SalePrice'] #to get correlation for specific field with other atributes

,SalePrice,BuildingAge,Area,Floor,Location
SalePrice,1.000000,0.447875,0.697199,0.336729,0.010615
BuildingAge,0.447875,1.000000,0.021282,0.260568,0.005180
Area,0.697199,0.021282,1.000000,0.179491,0.017981
Floor,0.336729,0.260568,0.179491,1.000000,0.005355
Location,0.010615,0.005180,0.017981,0.005355,1.000000


In [144]:
#selection to attributes where according to the correlation
Xtest = df[['BuildingAge','Area','Floor','Location']] 
X = df[['BuildingAge','Area','Floor']]
y = df[['SalePrice']]

from sklearn.feature_selection import RFE

rfe = RFE(regressor,n_features_to_select=3,step=43) # you can tell the eliminator how many features you need
fit = rfe.fit(Xtest, y) # let him calculate it free way
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_) #False means never to exclude it
print("Feature Ranking: %s" % fit.ranking_) #the more value the more will impact your model

for i in range(Xtest.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Num Features: 3
Selected Features: [ True False  True  True]
Feature Ranking: [1 2 1 1]
Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 2.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [147]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
chiSelector = SelectKBest(chi2,k=3)
X_kbest = chiSelector.fit(Xtest, y)
print(X_kbest.scores_)#We choose the highest values, which will be the first 3

[1.18453147e+02 5.61716913e+05 9.62733118e+03 1.10589874e+02]


In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [113]:
print(regressor.intercept_)
print(regressor.coef_)
print (regressor.predict([[1995,280,10]]))#prediction for apartment, with building age 1995, area 280, and floor 10

[-9564555.49820002]
[[4790.06562598  185.23478385 1305.25374555]]
[[56543.70256831]]


# import pickle and forecasting from saved model

In [114]:
import pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(regressor, file)

In [126]:
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict([[2022,400,15]])
Ypredict

array([[214629.91725946]])

In [116]:
Ypredict= pickle_model.predict([[2022,300,10]])
Ypredict

array([[189580.17014682]])